### GRIB nearest gridpoint

First we ensure the example file is available.

In [6]:
from earthkit.data import download_example_file, from_source
from earthkit.data.geo import nearest_point_haversine

In [7]:
ds = from_source("file", "test.grib")

#### Single refrence point

We define a **reference point** and get the index of the nearest gridpoint from the first field.

In [8]:
latlon = ds[0].to_latlon()
lat = latlon["lat"]
lon = latlon["lon"]

p_ref = (51.45, -0.97)
# distance is in m
idx, distance = nearest_point_haversine(p_ref, (lat, lon))
idx, distance

(array([102]), array([218417.94491761]))

With the resulting index we can get the value at the nearest gridpoint.

In [9]:
v = ds[0].values[idx]
v

array([280.68066406])

#### Multiple reference points

This time we use 3 reference points. 

In [10]:
p_ref = [
    [51.45, 44.49, 50.73], 
    [-0.97, 18.34, -17.1]
]
idx, dist = nearest_point_haversine(p_ref, (lat, lon))
idx

array([102, 144, 116])

In [11]:
v = ds[0].values[idx]
v

array([280.68066406, 296.4765625 , 285.05761719])

### 